# Function Calling Tutorial PT 2

Okay in this example I'm going to ask for the weather in both Sydney and Brazil so we need to ensure we add both of the function calls to the messages. Hence need to change my loop.

In [6]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv
import requests

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [7]:
# Define the function you are going to use:
# Uses Open-Meteo's Weather Forecast API!!!
def get_weather(latitude, longitude):
    url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={latitude}&longitude={longitude}"
        f"&current=temperature_2m"
    )
    
    response = requests.get(url)
    data = response.json()
    
    return data.get('current').get('temperature_2m')

# example use
get_weather("-33.8727", "151.2057")

14.6

In [ ]:
# define list of callable tools for model
tools = [
    {
        "type": "function",
        "name": "get_current_weather",
        "description": "Get the current weather in degrees Celcius for a given latitude and longitude",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "string",
                    "description": "The latitude of a place",
                },
                "longitude": {
                    "type": "string",
                    "description": "The longitude of a place",
                },
            },
            "required": ["latitude", "longitude"],
        }
    }
]

messages = [{"role": "user", "content": "What is the weather in Sydney and Busan today?"}]

#for lesson 3 function calling: if we try the query below this code wouldn't work
#messages = [{"role": "user", "content": "What is the weather and time in Sydney and Busan?"}]

response = client.responses.create(
    model="gpt-3.5-turbo",
    tools=tools,
    input=messages,
    tool_choice="auto",
)

#print(response.model_dump_json(indent=2))
print(response.output)

# WHAT HAPPENED?
# We called the api endpoint and it returned which tool we should use and the inputs

[ResponseFunctionToolCall(arguments='{"latitude":"-33.8688","longitude":"151.2093"}', call_id='call_YW6jdcXwH1W6nc0oSNgRFZEZ', name='get_current_weather', type='function_call', id='fc_053f9f8a5579bf140068c8318215548197bd073691d0400cd2', status='completed'), ResponseFunctionToolCall(arguments='{"latitude":"35.1796","longitude":"129.0756"}', call_id='call_V3PHIe5E7sYzEUMremium6Qc', name='get_current_weather', type='function_call', id='fc_053f9f8a5579bf140068c83182bf2881978a3ee45f59c3cf0f', status='completed')]


In [9]:
function_call = None
function_call_arguments = None
messages += response.output

for item in response.output:
    if item.type == "function_call":
        function_call = item
        function_call_arguments = json.loads(item.arguments)

        print(function_call)
        print(function_call_arguments)

        # 3. Execute the function logic for get_weather
        result = {"weather": get_weather(function_call_arguments["latitude"], 
                                        function_call_arguments["longitude"])}
        print(result)

        # 4. Provide function call results to the model
        messages.append({
            "type": "function_call_output",
            "call_id": function_call.call_id,
            "output": json.dumps(result),
        })

        from pprint import pprint
        pprint(messages)


ResponseFunctionToolCall(arguments='{"latitude":"-33.8688","longitude":"151.2093"}', call_id='call_YW6jdcXwH1W6nc0oSNgRFZEZ', name='get_current_weather', type='function_call', id='fc_053f9f8a5579bf140068c8318215548197bd073691d0400cd2', status='completed')
{'latitude': '-33.8688', 'longitude': '151.2093'}
{'weather': 14.2}
[{'content': 'What is the weather in Sydney and Busan today?', 'role': 'user'},
 ResponseFunctionToolCall(arguments='{"latitude":"-33.8688","longitude":"151.2093"}', call_id='call_YW6jdcXwH1W6nc0oSNgRFZEZ', name='get_current_weather', type='function_call', id='fc_053f9f8a5579bf140068c8318215548197bd073691d0400cd2', status='completed'),
 ResponseFunctionToolCall(arguments='{"latitude":"35.1796","longitude":"129.0756"}', call_id='call_V3PHIe5E7sYzEUMremium6Qc', name='get_current_weather', type='function_call', id='fc_053f9f8a5579bf140068c83182bf2881978a3ee45f59c3cf0f', status='completed'),
 {'call_id': 'call_YW6jdcXwH1W6nc0oSNgRFZEZ',
  'output': '{"weather": 14.2}',
  

In [10]:
response = client.responses.create(
    model="gpt-3.5-turbo",
    tools=tools,
    input=messages,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.output_text + "\n\n")
print(response.model_dump_json(indent=2))


Final output:
The current weather in Sydney is 14.2°C and in Busan is 23.0°C.


{
  "id": "resp_053f9f8a5579bf140068c83186acec819786cb528e38537928",
  "created_at": 1757950342.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-3.5-turbo-0125",
  "object": "response",
  "output": [
    {
      "id": "msg_053f9f8a5579bf140068c8318797f481979a8137e0803057ec",
      "content": [
        {
          "annotations": [],
          "text": "The current weather in Sydney is 14.2°C and in Busan is 23.0°C.",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_current_weather",
      "parameters": {
        "type": "object",
        "properties": {
          "latitude": {
            "type": "string",
            "d